In [1]:
# !pip install transformers -q
# !pip install sentencepiece -q

     |████████████████████████████████| 2.0MB 6.6MB/s 
     |████████████████████████████████| 3.2MB 23.0MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 
     |████████████████████████████████| 1.2MB 5.3MB/s 


In [2]:
import os
import json

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

SEED = 42

In [3]:
tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recomended.')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Using TPU


In [4]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 6
bsize = 32
BATCH_SIZE = bsize * strategy.num_replicas_in_sync
MAX_LEN = 256 # 512

In [ ]:
# def init(n, model):
#   df = pd.read_pickle('wsb_no_topics.pkl')

#   with open(f'wsb_topics_{n}.json', 'r') as fp:
#     topics = json.load(fp)

#   df["topic"] = -1

#   for i in range(0, len(topics)):
#       df.loc[topics[str(i)], "topic"] = i

#   n_classes = len(topics)

In [ ]:
# df = pd.read_pickle('wsb_no_topics.pkl')

# with open(f'wsb_topics_{n}.json', 'r') as fp:
#   topics = json.load(fp)

# df["topic"] = -1

# for i in range(0, len(topics)):
#     df.loc[topics[str(i)], "topic"] = i

# n_classes = len(topics)

In [ ]:
# AUTO = tf.data.experimental.AUTOTUNE

# # Configuration
# EPOCHS = 6
# bsize = 32
# BATCH_SIZE = bsize * strategy.num_replicas_in_sync
# MAX_LEN = 256 # 512
# MODEL = my_models[3]

# print('Selected model: ', MODEL)

In [ ]:
# X = df.selftext.values
# y = df.topic.values

# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df.topic, test_size=0.2, random_state=SEED)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=SEED)

In [5]:
def regular_encode(texts, tokenizer, maxlen=MAX_LEN):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        return_attention_mask=False, 
        return_token_type_ids=False,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [6]:
def build_model(transformer, n_classes, loss='sparse_categorical_crossentropy', max_len=MAX_LEN):
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    out = tf.keras.layers.Dense(n_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=input_word_ids, outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss=loss, metrics=['accuracy'])
    return model

In [7]:
def create_datasets(X_train, X_val, X_test, y_train, y_val, y_test, MODEL):
  tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL)

  x_train = regular_encode(X_train.tolist(), tokenizer, maxlen=MAX_LEN)
  x_valid = regular_encode(X_val.tolist(), tokenizer, maxlen=MAX_LEN)
  x_test = regular_encode(X_test.tolist(), tokenizer, maxlen=MAX_LEN)
  y_train = y_train.tolist()
  y_valid = y_val.tolist()
  y_test = y_test.tolist()


  train_dataset = (
      tf.data.Dataset
      .from_tensor_slices((x_train, y_train))
      .repeat()
      .shuffle(2048, seed=SEED)
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
  )

  valid_dataset = (
      tf.data.Dataset
      .from_tensor_slices((x_valid, y_valid))
      .batch(BATCH_SIZE)
      .cache()
      .prefetch(AUTO)
  )

  test_dataset = (
      tf.data.Dataset
      .from_tensor_slices(x_test)
      .batch(BATCH_SIZE)
  )

  n_steps = n_steps = x_train.shape[0] // BATCH_SIZE

  return train_dataset, valid_dataset, test_dataset, n_steps

In [15]:
def main(n):
  df = pd.read_pickle('relationships_no_labels.pkl')

  with open(f'relationships_topics_{n}.json', 'r') as fp:
    topics = json.load(fp)

  df["topic"] = -1

  for i in range(0, len(topics)):
      df.loc[topics[str(i)], "topic"] = i

  n_classes = len(topics)

  X = df.selftext.values
  y = df.topic.values

  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df.topic, test_size=0.2, random_state=SEED)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=SEED)

  train_dataset, valid_dataset, test_dataset, n_steps = create_datasets(X_train, X_val, X_test, y_train, y_val, y_test, MODEL)

  with strategy.scope():
    transformer_layer = transformers.TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, n_classes, max_len=MAX_LEN)

  train_history = model.fit(
      train_dataset,
      steps_per_epoch=n_steps,
      validation_data=valid_dataset,
      epochs=EPOCHS
  )

  y_pred = model.predict(test_dataset, verbose=1)

  predictions = [np.argmax(p) for p in y_pred]
  print(f'model: {MODEL}, epochs: {EPOCHS}, sample_len: {MAX_LEN}, batch_size: {bsize}')
  print(classification_report(y_test, predictions))

In [9]:
my_models = ['bert-base-uncased', 'roberta-base', 'jplu/tf-xlm-roberta-base',
             'distilbert-base-uncased-finetuned-sst-2-english']

MODEL = my_models[0]

In [17]:
main(5)

main(10)

main(15)

main(20)

main(25)

main(30)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
232/232 [==============================] - 181s 461ms/step - loss: 1.1010 - accuracy: 0.5427 - val_loss: 0.6095 - val_accuracy: 0.7557
Epoch 2/6
232/232 [==============================] - 93s 403ms/step - loss: 0.6414 - accuracy: 0.7500 - val_loss: 0.5794 - val_accuracy: 0.7682
Epoch 3/6
232/232 [==============================] - 93s 403ms/step - loss: 0.5770 - accuracy: 0.7736 - val_loss: 0.5689 - val_accuracy: 0.7729
Epoch 4/6
232/232 [==============================] - 93s 402ms/step - loss: 0.5348 - accuracy: 0.7922 - val_loss: 0.5706 - val_accuracy: 0.7744
Epoch 5/6
232/232 [==============================] - 93s 403ms/step - loss: 0.4940 - accuracy: 0.8082 - val_loss: 0.5814 - val_accuracy: 0.7737
Epoch 6/6
73/73 [==============================] - 23s 262ms/step
model: bert-base-uncased, epochs: 6, sample_len: 256, batch_size: 32
              precision    recall  f1-score   support

           0       0.77      0.80      0.78      4458
           1       0.68      0.77  

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
232/232 [==============================] - 184s 461ms/step - loss: 1.7297 - accuracy: 0.3864 - val_loss: 0.9616 - val_accuracy: 0.6540
Epoch 2/6
232/232 [==============================] - 94s 404ms/step - loss: 0.9686 - accuracy: 0.6518 - val_loss: 0.9002 - val_accuracy: 0.6757
Epoch 3/6
232/232 [==============================] - 94s 406ms/step - loss: 0.8666 - accuracy: 0.6873 - val_loss: 0.8836 - val_accuracy: 0.6845
Epoch 4/6
232/232 [==============================] - 93s 403ms/step - loss: 0.8089 - accuracy: 0.7088 - val_loss: 0.8382 - val_accuracy: 0.6969
Epoch 5/6
232/232 [==============================] - 93s 403ms/step - loss: 0.7688 - accuracy: 0.7225 - val_loss: 0.8582 - val_accuracy: 0.7007
Epoch 6/6
73/73 [==============================] - 23s 262ms/step
model: bert-base-uncased, epochs: 6, sample_len: 256, batch_size: 32
              precision    recall  f1-score   support

           0       0.71      0.76      0.74      3072
           1       0.71      0.78  

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If you

Epoch 1/6
232/232 [==============================] - 186s 466ms/step - loss: 2.4772 - accuracy: 0.2709 - val_loss: 1.4216 - val_accuracy: 0.5545
Epoch 2/6
232/232 [==============================] - 93s 402ms/step - loss: 1.4706 - accuracy: 0.5391 - val_loss: 1.3186 - val_accuracy: 0.5801
Epoch 3/6
232/232 [==============================] - 93s 403ms/step - loss: 1.3209 - accuracy: 0.5786 - val_loss: 1.2921 - val_accuracy: 0.5856
Epoch 4/6
232/232 [==============================] - 93s 403ms/step - loss: 1.2369 - accuracy: 0.6013 - val_loss: 1.2817 - val_accuracy: 0.5890
Epoch 5/6
232/232 [==============================] - 93s 403ms/step - loss: 1.1565 - accuracy: 0.6265 - val_loss: 1.2640 - val_accuracy: 0.5962
Epoch 6/6
73/73 [==============================] - 23s 265ms/step
model: bert-base-uncased, epochs: 6, sample_len: 256, batch_size: 32
              precision    recall  f1-score   support

           0       0.69      0.31      0.43       825
           1       0.56      0.47  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If you

Epoch 1/6
232/232 [==============================] - 185s 463ms/step - loss: 2.6434 - accuracy: 0.2374 - val_loss: 1.5612 - val_accuracy: 0.5137
Epoch 2/6
232/232 [==============================] - 93s 402ms/step - loss: 1.5955 - accuracy: 0.5086 - val_loss: 1.4643 - val_accuracy: 0.5377
Epoch 3/6
232/232 [==============================] - 93s 402ms/step - loss: 1.4364 - accuracy: 0.5532 - val_loss: 1.3838 - val_accuracy: 0.5590
Epoch 4/6
232/232 [==============================] - 93s 402ms/step - loss: 1.3393 - accuracy: 0.5772 - val_loss: 1.3625 - val_accuracy: 0.5691
Epoch 5/6
232/232 [==============================] - 93s 403ms/step - loss: 1.2571 - accuracy: 0.6024 - val_loss: 1.3489 - val_accuracy: 0.5721
Epoch 6/6
73/73 [==============================] - 23s 266ms/step
model: bert-base-uncased, epochs: 6, sample_len: 256, batch_size: 32
              precision    recall  f1-score   support

           0       0.56      0.36      0.44       639
           1       0.49      0.45  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#building the model on tpu
with strategy.scope():
    transformer_layer = transformers.TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
    
model.summary()

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['dropout_19', 'pre_classifier', 'classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 256)]             0         
_________________________________________________________________
tf_distil_bert_model_4 (TFDi TFBaseModelOutput(last_hi 66362880  
_________________________________________________________________
tf.__operators__.getitem_4 ( (None, 768)               0         
_________________________________________________________________
dropout_136 (Dropout)        (None, 768)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 3845      
Total params: 66,366,725
Trainable params: 66,366,725
Non-trainable params: 0
_________________________________________________________________
CPU times: user 5.89 s, sys: 3.42 s, total: 9.32 s
Wall time: 22.7 s


In [ ]:
x_train = regular_encode(X_train.tolist(), tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(X_val.tolist(), tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(X_test.tolist(), tokenizer, maxlen=MAX_LEN)
y_train = y_train.tolist()
y_valid = y_val.tolist()
y_test = y_test.tolist()

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048, seed=SEED)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL)

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/6
114/114 [==============================] - 86s 320ms/step - loss: 1.4216 - accuracy: 0.3894 - val_loss: 0.8938 - val_accuracy: 0.6513
Epoch 2/6
114/114 [==============================] - 25s 220ms/step - loss: 0.9207 - accuracy: 0.6417 - val_loss: 0.7187 - val_accuracy: 0.7239
Epoch 3/6
114/114 [==============================] - 25s 220ms/step - loss: 0.7381 - accuracy: 0.7157 - val_loss: 0.6775 - val_accuracy: 0.7381
Epoch 4/6
114/114 [==============================] - 25s 220ms/step - loss: 0.6463 - accuracy: 0.7526 - val_loss: 0.6260 - val_accuracy: 0.7573
Epoch 5/6
114/114 [==============================] - 25s 220ms/step - loss: 0.5661 - accuracy: 0.7863 - val_loss: 0.6143 - val_accuracy: 0.7627
Epoch 6/6
114/114 [==============================] - 25s 221ms/step - loss: 0.5144 - accuracy: 0.8039 - val_loss: 0.6016 - val_accuracy: 0.7738


In [ ]:
y_pred = model.predict(test_dataset, verbose=1)

36/36 [==============================] - 3s 63ms/step


In [ ]:
predictions = [np.argmax(p) for p in y_pred]
print(f'model: {MODEL}, epochs: {EPOCHS}, sample_len: {MAX_LEN}, batch_size: {bsize}')
print(classification_report(y_test, predictions))

model: distilbert-base-uncased-finetuned-sst-2-english, epochs: 6, sample_len: 256, batch_size: 32
              precision    recall  f1-score   support

           0       0.75      0.67      0.71      1455
           1       0.71      0.78      0.74      2331
           2       0.82      0.86      0.84      2872
           3       0.78      0.66      0.72      1704
           4       0.71      0.73      0.72       810

    accuracy                           0.76      9172
   macro avg       0.75      0.74      0.74      9172
weighted avg       0.76      0.76      0.76      9172



In [ ]:
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt

# encoded_classes = list(range(20))
# test_topics = y_test
# pred_topics = predictions = [np.argmax(p) for p in y_pred]
# confusion_mat = confusion_matrix(y_true = test_topics, y_pred = pred_topics, labels=list(encoded_classes))
# df_cm = pd.DataFrame(confusion_mat, index = list(encoded_classes),columns = list(encoded_classes))
# plt.rcParams['figure.figsize'] = (10,10)
# sns.heatmap(df_cm)